# 2-7 学習と検証の実施

## プログラムに実装
前節までに実装した内容を用いて学習プログラムを実装する．
流れは次の通り．

1. DataLoader の実装
1. ネットワークモデルの作成
1. 損失関数の定義
1. 最適化手法の設定
1. 学習・検証の実施

## DataLoader
2.2，2.3節の実装を用いる．

In [1]:
import time
import pandas as pd
import torch
import torch.utils.data as data
from torch import nn
from utils.dataloader import od_collate_fn, make_VOC_dataloader
from utils.dataset import make_datapath_list, VOCDataset, DataTransform, Anno_xml2list
from utils.forward import SSD
from utils.loss import MultiBoxLoss

In [2]:
# ファイルパスのリストを作成
rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath)

# Dataset を作成
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair','cow', 'diningtable', 
               'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)  # BGR 色平均
input_size = 300              # 画像の入力サイズは 300x300

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train",
                           transform=DataTransform(input_size, color_mean),
                           transform_anno=Anno_xml2list(voc_classes))
val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val",
                           transform=DataTransform(input_size, color_mean),
                           transform_anno=Anno_xml2list(voc_classes))

# データローダの作成
batch_size = 8
train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn)
val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn)

# 辞書型のオブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

## ネットワークモデルの作成
2.4節で実装したネットワークを用いる．
vgg モジュールの初期値には ImageNet で事前に学習した結合パラメータを用いる．
ここでは，weights/vgg16_reducedfc.pth にダウンロード済みのものを用いる．
それ以外のモジュールの初期値は He の初期値を用いる．
He の初期値は，入力チャネル数 input_n に対して結合パラメータの初期値に，平均0，分散 2/input_n のガウス分布に従う乱数を使用する．

In [3]:
# SSD300 の設定
ssd_cfg = {
    'num_classes': 21, # 背景クラスを含めた合計クラス数
    'input_size': 300, # 画像の入力サイズ
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],      # 出力するデフォルトボックスのアスペクト比の種類
    'feature_maps': [38, 19, 10, 5, 3, 1],      # 各 source の画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],         # デフォルトボックスの大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264],  # デフォルトボックスの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315], # デフォルトボックスの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]]
}

# SSD ネットワークモデル
net = SSD(phase="train", cfg=ssd_cfg)

# SSD の結合パラメータの初期値を設定
# vgg モジュールの初期値をロード
vgg_weights = torch.load("./weights/vgg16_reducedfc.pth")
net.vgg.load_state_dict(vgg_weights)

# SSD のその他のネットワークの重みは He の初期値で初期化
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight.data)
        if m.bias is not None:  # バイアス項がある場合
            nn.init.constant_(m.bias, 0.0)
            
# He の初期値を適用
net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)

# GPU が使えるかを確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)
print("ネットワーク設定完了：学習済みの重みをロードしました")

使用デバイス： cuda:0
ネットワーク設定完了：学習済みの重みをロードしました


## 損失関数と最適化手法の設定

In [4]:
# 損失関数の設定
criterion = MultiBoxLoss(jaccard_threshold=0.5, neg_pos=3, device=device)
# 最適化手法の設定
optimizer = torch.optim.SGD(net.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-4)

## 学習と検証の実施
学習と検証を行う関数を定義する．
10 epoch ごとに検証を行い，各 epoch ごとに学習と検証の loss の値を log_output.csv に保存する．
ネットワークの結合パラメータも 10 epoch ごとに保存する．

In [5]:
# モデルを学習させる関数を作成
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    # GPU が使えるか確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    
    # ネットワークを GPU へ
    net.to(device)
    
    # ネットワークがある程度固定であれば高速化
    torch.backends.cudnn.benchmark = True
    
    # イテレーションのカウンタをセット
    iteration = 1
    epoch_train_loss = 0.0  # 学習時の epoch の損失和
    epoch_val_loss = 0.0    # 検証時の epoch の損失和
    logs = []
    
    # epoch のグループ
    for epoch in range(num_epochs + 1):
        t_epoch_start = time.time()
        t_iter_start = time.time()
        
        print("-------------------")
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print("-------------------")
        
        # epoch ごとの訓練と検証のループ
        for phase in ["train", "val"]:
            if phase == "train":
                net.train()  # モデルを訓練モードに設定
                print("-------------------")
                print(" (train) ")
            else:
                if (epoch + 1) % 10 == 0:
                    net.eval()  # モデルを検証モードに設定
                    print("-------------------")
                    print(" (val) ")
                else:
                    continue
            
            # データローダから mini batch サイズずつ取り出すループ
            for images, targets in dataloaders_dict[phase]:
                # GPU が使用可能であれば GPU にデータを送る
                images = images.to(device)
                targets = [ann.to(device) for ann in targets] # リストの各要素のテンソルを GPU に送る
                
                # optimizer を初期化
                optimizer.zero_grad()
                
                # 学習の実行
                with torch.set_grad_enabled(phase == 'train'):
                    # 順伝播の計算
                    outputs = net(images)
                    
                    # 損失の計算
                    loss_l, loss_c = criterion(outputs, targets)
                    loss = loss_l + loss_c
                    
                    # 訓練時には逆伝播も計算
                    if phase == "train":
                        loss.backward()
                        # 勾配が大きくなりすぎると学習が不安定になるため最大で2.0となるように制限
                        nn.utils.clip_grad_value_(net.parameters(), clip_value=2.0)
                        # パラメータを更新
                        optimizer.step()
                        # 10 iter に1度 loss を表示
                        if iteration % 10 == 0:
                            t_iter_finish =time.time()
                            duration = t_iter_finish - t_iter_start
                            print('iteration {} || Loss: {:.4f} || 10iter:{:.4f} sec.'.format(iteration, loss.item(), duration))
                            t_iter_start = time.time()
                            
                        epoch_train_loss += loss.item()
                        iteration += 1
                        
                    # 検証時
                    else:
                        epoch_val_loss += loss.item()
                        
        # epoch の phase ごとの loss と正解率
        t_epoch_finish = time.time()
        print("-------------------")
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} || Epoch_VAL_Loss:{:.4f}'.format(epoch + 1, epoch_train_loss, epoch_val_loss))
        t_epoch_start = time.time()
        
        # ログの保存
        log_epoch = {"epoch": epoch + 1, "train_loss": epoch_train_loss, "val_loss": epoch_val_loss}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output.csv")
        
        # 損失値のリセット
        epoch_train_loss = 0.0
        epoch_val_loss = 0.0
        
        # 10 epoch ごとにネットワークを保存する
        if (epoch + 1) % 10 == 0:
            torch.save(net.state_dict(), 'weights/ssd300_' + str(epoch+1) + '.pth')

In [6]:
num_epochs = 50
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

使用デバイス： cuda:0
-------------------
Epoch 1/50
-------------------
-------------------
 (train) 
iteration 10 || Loss: 15.2293 || 10iter:11.3841 sec.
iteration 20 || Loss: 13.6413 || 10iter:7.8792 sec.
iteration 30 || Loss: 14.4254 || 10iter:7.7383 sec.
iteration 40 || Loss: 9.7031 || 10iter:7.7602 sec.
iteration 50 || Loss: 11.6924 || 10iter:7.7592 sec.
iteration 60 || Loss: 10.1063 || 10iter:7.7779 sec.
iteration 70 || Loss: 10.8501 || 10iter:7.8928 sec.
iteration 80 || Loss: 7.9851 || 10iter:7.8083 sec.
iteration 90 || Loss: 9.8848 || 10iter:8.0706 sec.
iteration 100 || Loss: 9.9436 || 10iter:8.2000 sec.
iteration 110 || Loss: 8.4661 || 10iter:7.9685 sec.
iteration 120 || Loss: 8.2492 || 10iter:7.8529 sec.
iteration 130 || Loss: 9.4068 || 10iter:8.0069 sec.
iteration 140 || Loss: 7.9618 || 10iter:7.8789 sec.
iteration 150 || Loss: 9.2625 || 10iter:7.8460 sec.
iteration 160 || Loss: 10.8993 || 10iter:7.9483 sec.
iteration 170 || Loss: 8.4994 || 10iter:8.1447 sec.
iteration 180 || Loss

iteration 1490 || Loss: 6.5225 || 10iter:8.0989 sec.
iteration 1500 || Loss: 7.7073 || 10iter:7.9287 sec.
iteration 1510 || Loss: 7.0649 || 10iter:8.2696 sec.
iteration 1520 || Loss: 6.3736 || 10iter:8.0390 sec.
iteration 1530 || Loss: 6.4012 || 10iter:8.2679 sec.
iteration 1540 || Loss: 8.5400 || 10iter:8.0381 sec.
iteration 1550 || Loss: 8.2234 || 10iter:7.9244 sec.
iteration 1560 || Loss: 8.3384 || 10iter:7.8214 sec.
iteration 1570 || Loss: 8.7888 || 10iter:7.9256 sec.
iteration 1580 || Loss: 8.1114 || 10iter:7.8121 sec.
iteration 1590 || Loss: 6.9493 || 10iter:7.9816 sec.
iteration 1600 || Loss: 8.8552 || 10iter:7.8491 sec.
iteration 1610 || Loss: 6.4098 || 10iter:7.7929 sec.
iteration 1620 || Loss: 10.6343 || 10iter:7.8728 sec.
iteration 1630 || Loss: 9.3367 || 10iter:7.9640 sec.
iteration 1640 || Loss: 7.4395 || 10iter:7.9645 sec.
iteration 1650 || Loss: 9.3289 || 10iter:7.9667 sec.
iteration 1660 || Loss: 9.3559 || 10iter:7.9214 sec.
iteration 1670 || Loss: 9.8283 || 10iter:7.88

iteration 2970 || Loss: 6.7068 || 10iter:7.9372 sec.
iteration 2980 || Loss: 8.8598 || 10iter:8.0227 sec.
iteration 2990 || Loss: 8.4985 || 10iter:7.9660 sec.
iteration 3000 || Loss: 6.6875 || 10iter:7.9178 sec.
iteration 3010 || Loss: 7.6920 || 10iter:7.8602 sec.
iteration 3020 || Loss: 7.0075 || 10iter:7.8251 sec.
iteration 3030 || Loss: 9.7056 || 10iter:7.9332 sec.
iteration 3040 || Loss: 6.4420 || 10iter:7.8441 sec.
iteration 3050 || Loss: 8.9670 || 10iter:7.8265 sec.
iteration 3060 || Loss: 5.1964 || 10iter:7.8961 sec.
iteration 3070 || Loss: 9.4408 || 10iter:7.9173 sec.
iteration 3080 || Loss: 7.6450 || 10iter:7.8456 sec.
iteration 3090 || Loss: 10.1844 || 10iter:7.8211 sec.
iteration 3100 || Loss: 6.3175 || 10iter:7.8938 sec.
iteration 3110 || Loss: 9.1922 || 10iter:7.8135 sec.
iteration 3120 || Loss: 7.8163 || 10iter:7.8980 sec.
iteration 3130 || Loss: 6.9840 || 10iter:7.7785 sec.
iteration 3140 || Loss: 6.4152 || 10iter:7.8841 sec.
iteration 3150 || Loss: 7.1860 || 10iter:7.83

iteration 4460 || Loss: 6.0655 || 10iter:7.9440 sec.
iteration 4470 || Loss: 7.5246 || 10iter:7.9462 sec.
iteration 4480 || Loss: 7.0708 || 10iter:7.8630 sec.
iteration 4490 || Loss: 8.2861 || 10iter:7.9180 sec.
iteration 4500 || Loss: 8.7318 || 10iter:7.8754 sec.
iteration 4510 || Loss: 7.2961 || 10iter:7.7836 sec.
iteration 4520 || Loss: 7.0793 || 10iter:7.7525 sec.
iteration 4530 || Loss: 7.1745 || 10iter:7.8311 sec.
iteration 4540 || Loss: 6.6808 || 10iter:7.8763 sec.
iteration 4550 || Loss: 6.1350 || 10iter:7.8630 sec.
iteration 4560 || Loss: 6.9599 || 10iter:7.9714 sec.
iteration 4570 || Loss: 5.9796 || 10iter:7.8952 sec.
iteration 4580 || Loss: 6.0352 || 10iter:7.8581 sec.
iteration 4590 || Loss: 6.7193 || 10iter:7.8452 sec.
iteration 4600 || Loss: 6.4083 || 10iter:7.9866 sec.
iteration 4610 || Loss: 6.5827 || 10iter:7.8003 sec.
iteration 4620 || Loss: 6.7247 || 10iter:7.8533 sec.
iteration 4630 || Loss: 6.0411 || 10iter:7.8736 sec.
iteration 4640 || Loss: 7.6450 || 10iter:7.902

iteration 5950 || Loss: 6.6091 || 10iter:7.8000 sec.
iteration 5960 || Loss: 6.3295 || 10iter:7.8808 sec.
iteration 5970 || Loss: 6.2572 || 10iter:7.9762 sec.
iteration 5980 || Loss: 5.5530 || 10iter:7.8583 sec.
iteration 5990 || Loss: 6.0298 || 10iter:7.8700 sec.
iteration 6000 || Loss: 7.1839 || 10iter:7.8505 sec.
iteration 6010 || Loss: 5.3397 || 10iter:7.8919 sec.
iteration 6020 || Loss: 7.2483 || 10iter:7.8508 sec.
iteration 6030 || Loss: 5.5321 || 10iter:7.8559 sec.
iteration 6040 || Loss: 6.4940 || 10iter:7.7316 sec.
iteration 6050 || Loss: 7.8727 || 10iter:7.8447 sec.
iteration 6060 || Loss: 5.4328 || 10iter:7.9060 sec.
iteration 6070 || Loss: 6.0718 || 10iter:7.7242 sec.
iteration 6080 || Loss: 6.2748 || 10iter:7.7754 sec.
iteration 6090 || Loss: 5.6395 || 10iter:7.8277 sec.
iteration 6100 || Loss: 5.1661 || 10iter:7.9305 sec.
iteration 6110 || Loss: 4.8291 || 10iter:7.8224 sec.
iteration 6120 || Loss: 5.2806 || 10iter:7.8562 sec.
iteration 6130 || Loss: 7.6413 || 10iter:7.820

...

iteration 29340 || Loss: 2.9244 || 10iter:7.9814 sec.
iteration 29350 || Loss: 4.8834 || 10iter:7.9309 sec.
iteration 29360 || Loss: 3.4122 || 10iter:7.8755 sec.
iteration 29370 || Loss: 3.6185 || 10iter:7.8402 sec.
iteration 29380 || Loss: 3.9857 || 10iter:7.9016 sec.
iteration 29390 || Loss: 4.2109 || 10iter:7.9797 sec.
iteration 29400 || Loss: 3.1566 || 10iter:7.9608 sec.
iteration 29410 || Loss: 4.1209 || 10iter:8.0315 sec.
iteration 29420 || Loss: 3.9366 || 10iter:7.9312 sec.
iteration 29430 || Loss: 4.1159 || 10iter:7.9787 sec.
iteration 29440 || Loss: 3.8136 || 10iter:7.8219 sec.
iteration 29450 || Loss: 4.5473 || 10iter:7.9792 sec.
iteration 29460 || Loss: 5.2258 || 10iter:7.9849 sec.
iteration 29470 || Loss: 2.8274 || 10iter:7.8037 sec.
iteration 29480 || Loss: 4.0416 || 10iter:7.9455 sec.
iteration 29490 || Loss: 3.8087 || 10iter:7.9298 sec.
iteration 29500 || Loss: 3.7669 || 10iter:7.8347 sec.
iteration 29510 || Loss: 4.6277 || 10iter:7.8666 sec.
iteration 29520 || Loss: 5.1

iteration 30800 || Loss: 4.0112 || 10iter:7.8110 sec.
iteration 30810 || Loss: 3.8586 || 10iter:7.9054 sec.
iteration 30820 || Loss: 3.2176 || 10iter:7.7174 sec.
iteration 30830 || Loss: 3.9157 || 10iter:7.8640 sec.
iteration 30840 || Loss: 3.9153 || 10iter:7.8245 sec.
iteration 30850 || Loss: 4.1232 || 10iter:7.7755 sec.
iteration 30860 || Loss: 4.1392 || 10iter:7.8403 sec.
iteration 30870 || Loss: 4.6893 || 10iter:7.9559 sec.
iteration 30880 || Loss: 3.9300 || 10iter:7.8897 sec.
iteration 30890 || Loss: 2.7949 || 10iter:7.8382 sec.
iteration 30900 || Loss: 3.6569 || 10iter:7.9167 sec.
iteration 30910 || Loss: 4.1503 || 10iter:7.9000 sec.
iteration 30920 || Loss: 4.9536 || 10iter:7.8635 sec.
iteration 30930 || Loss: 4.5077 || 10iter:7.8586 sec.
iteration 30940 || Loss: 3.6928 || 10iter:7.9844 sec.
iteration 30950 || Loss: 3.4461 || 10iter:7.8855 sec.
iteration 30960 || Loss: 3.9630 || 10iter:7.9297 sec.
iteration 30970 || Loss: 3.7434 || 10iter:7.8932 sec.
iteration 30980 || Loss: 2.9

iteration 32260 || Loss: 3.3920 || 10iter:7.9343 sec.
iteration 32270 || Loss: 4.1701 || 10iter:7.9036 sec.
iteration 32280 || Loss: 3.8287 || 10iter:7.8800 sec.
iteration 32290 || Loss: 4.3346 || 10iter:7.9383 sec.
iteration 32300 || Loss: 3.5365 || 10iter:7.9164 sec.
iteration 32310 || Loss: 3.8773 || 10iter:7.9173 sec.
iteration 32320 || Loss: 3.5265 || 10iter:7.9178 sec.
iteration 32330 || Loss: 3.4438 || 10iter:7.8816 sec.
iteration 32340 || Loss: 4.6182 || 10iter:7.9908 sec.
iteration 32350 || Loss: 3.3058 || 10iter:7.8972 sec.
iteration 32360 || Loss: 3.6754 || 10iter:7.8277 sec.
iteration 32370 || Loss: 3.5025 || 10iter:7.9114 sec.
iteration 32380 || Loss: 3.4778 || 10iter:8.0007 sec.
iteration 32390 || Loss: 3.7568 || 10iter:7.9764 sec.
iteration 32400 || Loss: 4.7126 || 10iter:7.9222 sec.
iteration 32410 || Loss: 3.4332 || 10iter:7.9051 sec.
iteration 32420 || Loss: 3.8258 || 10iter:7.9598 sec.
iteration 32430 || Loss: 2.8636 || 10iter:7.9809 sec.
iteration 32440 || Loss: 2.4

iteration 33720 || Loss: 3.3330 || 10iter:8.0327 sec.
iteration 33730 || Loss: 3.4981 || 10iter:7.8459 sec.
iteration 33740 || Loss: 2.8920 || 10iter:7.7973 sec.
iteration 33750 || Loss: 4.4048 || 10iter:7.9839 sec.
iteration 33760 || Loss: 4.3375 || 10iter:7.9566 sec.
iteration 33770 || Loss: 3.7253 || 10iter:7.9883 sec.
iteration 33780 || Loss: 3.8667 || 10iter:7.8452 sec.
iteration 33790 || Loss: 3.3553 || 10iter:7.7922 sec.
iteration 33800 || Loss: 4.0714 || 10iter:7.8893 sec.
iteration 33810 || Loss: 4.8243 || 10iter:7.8637 sec.
iteration 33820 || Loss: 3.8792 || 10iter:7.9015 sec.
iteration 33830 || Loss: 4.4430 || 10iter:7.8341 sec.
iteration 33840 || Loss: 4.7149 || 10iter:7.9698 sec.
iteration 33850 || Loss: 4.6001 || 10iter:7.9268 sec.
iteration 33860 || Loss: 3.0143 || 10iter:7.9726 sec.
iteration 33870 || Loss: 3.7538 || 10iter:7.9294 sec.
iteration 33880 || Loss: 4.0569 || 10iter:7.9495 sec.
iteration 33890 || Loss: 3.4847 || 10iter:7.8391 sec.
iteration 33900 || Loss: 3.7

iteration 35180 || Loss: 3.4507 || 10iter:7.9795 sec.
iteration 35190 || Loss: 4.6380 || 10iter:7.8513 sec.
iteration 35200 || Loss: 3.2867 || 10iter:7.9942 sec.
iteration 35210 || Loss: 3.3933 || 10iter:8.0478 sec.
iteration 35220 || Loss: 3.4519 || 10iter:7.8348 sec.
iteration 35230 || Loss: 2.3306 || 10iter:7.9054 sec.
iteration 35240 || Loss: 3.7508 || 10iter:8.0127 sec.
iteration 35250 || Loss: 3.2243 || 10iter:7.9015 sec.
iteration 35260 || Loss: 3.7075 || 10iter:7.8579 sec.
iteration 35270 || Loss: 2.9057 || 10iter:7.8094 sec.
iteration 35280 || Loss: 3.8047 || 10iter:7.8066 sec.
iteration 35290 || Loss: 3.8987 || 10iter:7.9055 sec.
iteration 35300 || Loss: 3.5411 || 10iter:8.0266 sec.
iteration 35310 || Loss: 3.5757 || 10iter:7.9389 sec.
iteration 35320 || Loss: 3.4338 || 10iter:7.9155 sec.
iteration 35330 || Loss: 4.3760 || 10iter:7.8957 sec.
iteration 35340 || Loss: 4.5024 || 10iter:7.9921 sec.
iteration 35350 || Loss: 3.5631 || 10iter:7.8468 sec.
iteration 35360 || Loss: 4.3